This code was based on in the following references:

[1] "Grey Wolf Optimizer" published in 2013 by Seyedali Mirjalili, Seyed Mohammad Mirjalili e Andrew Lewis

[2] "A Novel Search Algorithm based on Fish School Behavior" published in 2008 by Bastos Filho, Lima Neto, Lins, D. O. Nascimento and P. Lima

[2] "An Enhanced Fish School Search Algorithm" published in 2013 by Bastos Filho and  D. O. Nascimento

[3] "Defining a Standard for Particle Swarm Optimization" published in 2007 by Bratton and Kennedy

### Imports

In [1]:
import os
import sys
import csv
import random
import copy
import math
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
from google.colab import drive, files
%matplotlib inline
plt.rcParams["figure.figsize"] = (13,8)

### Connection with Google Drive

In [2]:
drive.mount('/content/drive', force_remount=True) #  Connect to drive
FOLDERNAME = 'POLI/decimo/tcc/Algoritms/Algoritms Notebook/' #  Set folder that have the datasts
assert FOLDERNAME is not None, "[!] Enter the foldername."
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME)) #  Locale the foldeer

%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/POLI/decimo/tcc/Algoritms/Algoritms Notebook


### Class of Wolfs and Volitive Wolf Pack with each method to execute optimization

In [3]:
from ObjectiveFunction import *
from SearchSpaceInitializer import UniformSSInitializer, OneQuarterDimWiseSSInitializer
%cd ../

class Wolf(object):
  def __init__(self, dim):
    self.pos = np.zeros(dim)
    self.score = np.inf
    self.aromatic_intensity = 0
    self.delta_pos = np.zeros(dim)
    self.delta_score = 0
    self.best_score = 0
    self.best_pos = np.zeros(dim)
    self.last_pos = np.zeros(dim)

  def update_bests(self, score=None, pos=None):
    score = self.score if score is None else score
    pos = self.pos if pos is None else pos
    if score < self.best_score or self.best_score == np.inf:
      self.best_pos = pos
      self.best_score = score
  
  def is_wolf(self, w: 'Wolf'):
    check_pos = w.pos == self.pos
    if type(check_pos) is np.ndarray:
      check_pos = check_pos.sum() == len(self.pos)

    # check_last_pos = w.last_pos == self.last_pos
    # if type(check_last_pos) is np.ndarray:
    #   check_last_pos = check_last_pos.sum() == len(self.pos)
    
    # check_pos = check_last_pos and check_pos
    # print(check_pos, f"{w.score}-{self.score}", f"{w.aromatic_intensity}-{self.aromatic_intensity}")
    return check_pos and w.score == self.score and w.aromatic_intensity == self.aromatic_intensity
  
  def dist_between(self, w:'Wolf'):
    return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(self.pos, w.pos)))


class Alpha(Wolf):
  def __init__(self, dim, score, pos, last_pos, ai):
    self.score = score
    self.pos = pos
    self.last_pos = last_pos
    self.aromatic_intensity = ai


class Beta(Wolf):
  def __init__(self, dim, score, pos, last_pos, ai):
    self.score = score
    self.pos = pos
    self.last_pos = last_pos
    self.aromatic_intensity = ai


class Delta(Wolf):
  def __init__(self, dim, score, pos, last_pos, ai):
    self.score = score
    self.pos = pos
    self.last_pos = last_pos
    self.aromatic_intensity = ai


class VolitivePack(object):
  def __init__(self, objective_function, space_initializer, n_iter, pack_size,
               vol_init=0.01, vol_final=0.001, min_ai=0.4, analytic_in=False):
    self.objective_function = objective_function # função de avalição de custo
    self.space_initializer = space_initializer # posições iniciais dos peixes

    self.dim = objective_function.dim
    self.minf = objective_function.minf # limite minimo da função
    self.maxf = objective_function.maxf # limite máximo da função
    self.n_iter = n_iter

    self.pack_size = pack_size  # quantidade de peixes
    self.min_ai = min_ai
    self.prev_ai_pack = 0.0
    self.curr_ai_pack = 0.0
    self.barycenter = np.zeros(self.dim)

    self.a = 2
    self.r1 = np.random.uniform(size=self.dim)
    self.r2 = np.random.uniform(size=self.dim)
    self.a1 = 2 * self.a * self.r1 - self.a
    self.c1 = 2 * self.r2
    self.step_vol_init = vol_init
    self.step_vol_final = vol_final
    self.curr_step_vol = self.step_vol_init * (self.maxf - self.minf)
    self.curr_mult_vol = 0
    self.best = None
    self.alpha = None
    self.beta = None
    self.delta = None
    
    self.analytic_in = analytic_in
    self.i_net = np.zeros((n_iter, pack_size, pack_size))

    self.max_delta_score = 0
    self.optimum_score_tracking_iter = []
    self.optimum_posit_tracking_iter = []
    self.optimum_score_tracking_eval = []
  
  def __init_score_tracking(self):
    self.optimum_score_tracking_iter = []
    self.optimum_posit_tracking_iter = []
    self.optimum_score_tracking_eval = []

  def __gen_aromatic_intensity(self, score):
    return self.n_iter / score

  def __init_wolf(self, pos):
    wolf = Wolf(self.dim)
    wolf.pos = pos
    wolf.score = self.objective_function.evaluate(wolf.pos)
    wolf.aromatic_intensity = self.__gen_aromatic_intensity(wolf.score)
    self.optimum_score_tracking_eval.append(self.best.score)
    return wolf

  def __init_pack(self):
    self.best = Wolf(self.dim)
    self.alpha = Wolf(self.dim)
    self.beta = Wolf(self.dim)
    self.delta = Wolf(self.dim)
    self.pack = []
    self.curr_ai_pack = 0.0
    self.prev_ai_pack = 0.0
    
    positions = self.space_initializer.sample(self.objective_function, self.pack_size)

    for idx in range(self.pack_size):
      wolf = self.__init_wolf(positions[idx])
      self.pack.append(wolf)
      self.curr_ai_pack += wolf.aromatic_intensity
    self.prev_ai_pack = self.curr_ai_pack
    self.sniffing()
    self.update_hierarchy()
    # self.optimum_score_tracking_iter.append(self.alpha.score)

  def update_steps(self, curr_iter):
    self.a = 2 - curr_iter * (2 / self.n_iter)
    self.curr_step_vol = self.step_vol_init - curr_iter * float(self.step_vol_init - self.step_vol_final) / self.n_iter

  def update_variables(self):
    self.r1 = np.random.uniform(size=self.dim)
    self.r2 = np.random.uniform(size=self.dim)
    self.a1 = 2 * self.a * self.r1 - self.a
    self.c1 = 2 * self.r2
    
  def total_pack_ai(self):
    self.prev_ai_pack = self.curr_ai_pack
    self.curr_ai_pack = 0.0
    for wolf in self.pack:
      self.curr_ai_pack += wolf.aromatic_intensity

  def calculate_barycenter(self):
    barycenter = np.zeros((self.dim,), dtype=float)
    density = 0.0

    for wolf in self.pack:
      density += wolf.aromatic_intensity
      for dim in range(self.dim):
        barycenter[dim] += (wolf.pos[dim] * wolf.aromatic_intensity)
    for dim in range(self.dim):
      barycenter[dim] = barycenter[dim] / density

    return barycenter

  def update_hierarchy(self, remake=False, alpha=True, beta=True, delta=True):
    """
    Thid function is a copy of the original in github, but I don't if this is
    correct since here we have a volitive moviment and the alpha/beta/delta
    can be not exist more. 'remake' do the find the alpha/beta/delta again, and
    when False, the wolfes remeber whats the leaders even don't exist more
    """
    self.max_delta_score = 0
    if remake:
      self.alpha = Wolf(self.dim) if alpha else self.alpha
      self.beta = Wolf(self.dim) if beta else self.beta
      self.delta = Wolf(self.dim) if delta else self.delta
    
    for wolf in self.pack:
      # wolf.update_bests()
      if wolf.score < self.best.score:
        self.best = copy.deepcopy(wolf)
      if wolf.score < self.alpha.score:
        self.delta = copy.deepcopy(self.beta)
        self.beta = copy.deepcopy(self.alpha)
        self.alpha = copy.deepcopy(wolf)
        # self.delta = Delta(self.dim, self.beta.score, self.beta.pos, self.beta.last_pos, self.beta.aromatic_intensity)
        # self.beta = Beta(self.dim, self.alpha.score, self.alpha.pos, self.alpha.last_pos, self.alpha.aromatic_intensity)
        # self.alpha = Alpha(self.dim, wolf.score, wolf.pos, wolf.last_pos, wolf.aromatic_intensity)
      elif wolf.score > self.alpha.score and wolf.score < self.beta.score:
        self.delta = copy.deepcopy(self.beta)
        self.beta = copy.deepcopy(self.alpha)
      elif wolf.score > self.alpha.score and wolf.score > self.beta.score and wolf.score < self.alpha.score:
        self.delta = copy.deepcopy(self.beta)


  def sniffing(self):
    self.prev_ai_pack = self.curr_ai_pack
    self.curr_ai_pack = 0.0
    for wolf in self.pack:
      if self.max_delta_score:
        wolf.aromatic_intensity = wolf.aromatic_intensity + (wolf.delta_score / self.max_delta_score)
      if wolf.aromatic_intensity < self.min_ai:
        wolf.aromatic_intensity = self.min_ai
      
      self.curr_ai_pack += wolf.aromatic_intensity

  def collective_movement(self):
    for wolf in self.pack:
      new_pos = np.zeros((self.dim,), dtype=float)
      self.update_variables()
      d_alpha = abs(self.c1 * self.alpha.pos - wolf.pos)
      x1 = self.alpha.pos - self.a1 * d_alpha
      
      self.update_variables()
      d_beta = abs(self.c1 * self.beta.pos - wolf.pos)
      x2 = self.beta.pos - self.a1 * d_beta

      self.update_variables()
      d_delta = abs(self.c1 * self.delta.pos - wolf.pos)
      x3 = self.delta.pos - self.a1 * d_delta

      new_pos = (x1 + x2 + x3) / 3
      new_pos[new_pos < self.minf] = self.minf
      new_pos[new_pos > self.maxf] = self.maxf
      
      score = self.objective_function.evaluate(new_pos)
      # self.optimum_score_tracking_eval.append(self.alpha.score)
      # if score < wolf.score:
      wolf.delta_score = abs(score - wolf.score)
      delta_pos = np.zeros((self.dim,), dtype=float)
      for idx in range(self.dim):
        delta_pos[idx] = new_pos[idx] - wolf.pos[idx]
      wolf.last_pos = copy.copy(wolf.pos)
      wolf.score = score
      wolf.delta_pos = delta_pos
      wolf.pos = new_pos
      # else:
      #   wolf.delta_pos = np.zeros((self.dim,), dtype=float)
      #   wolf.delta_score = 0
      if wolf.delta_score > self.max_delta_score:
        self.max_delta_score = wolf.delta_score

  def collective_volitive_movement(self):
    # self.total_pack_ai()
    barycenter = self.calculate_barycenter()
    self.barycenter = barycenter
    for wolf in self.pack:
      new_pos = np.zeros((self.dim,), dtype=float)
      # for dim in range(self.dim):
      if self.curr_ai_pack > self.prev_ai_pack:
        self.curr_mult_vol = 1
        jump = self.curr_mult_vol * self.a
        new_pos = wolf.pos - ((wolf.pos - barycenter) * (self.curr_step_vol * jump) * np.random.uniform(0, 1, size=self.dim))
      else:
        self.curr_mult_vol += 10
        jump = self.curr_mult_vol * self.a
        new_pos = wolf.pos + ((wolf.pos - barycenter) * (self.curr_step_vol * jump) * np.random.uniform(0, 1, size=self.dim))
      new_pos[new_pos < self.minf] = self.minf
      new_pos[new_pos > self.maxf] = self.maxf

      # score = self.objective_function.evaluate(new_pos)
      # self.optimum_score_tracking_eval.append(self.alpha.score)
      # if self.analytic_in:
      #   wolf.last_pos = copy.copy(wolf.pos)
      # wolf.score = score
      wolf.pos = new_pos

  def get_analytic_in(self, n_iter, euclidean=True, volitive=True):
    interaction_in = []
    for w_i in self.pack:
      interaction_w = []
      is_leader = self.alpha.is_wolf(w_i) or self.beta.is_wolf(w_i) or self.delta.is_wolf(w_i)
      for w_j in self.pack:
        euclidean_distance = w_i.dist_between(w_j) if is_leader and euclidean else 0
        interaction_vol = 0
        if volitive:
          for d in range(self.dim):
            # contribuition or interaction can be negative?????
            contribuition = (w_i.aromatic_intensity * w_i.last_pos[d])/self.barycenter[d]
            interaction_vol += (w_j.delta_pos[d] * contribuition)
        i_net = euclidean_distance + interaction_vol
        interaction_w.append(i_net)
      interaction_in.append(interaction_w)
    self.i_net[n_iter] += interaction_in
    # print(n_iter, self.i_net[n_iter].sum())

  def optimize(self):
    self.__init_score_tracking()
    self.__init_pack()

    for i in range(self.n_iter):
      self.update_steps(i)
      self.collective_volitive_movement()
      self.sniffing()
      # self.update_hierarchy() # It's can be use lonely without the second (remake True is a little bit bad to Sphere and Schwefel, but good to Multimodal)
      if self.analytic_in:
        self.get_analytic_in(i, euclidean=True, volitive=False)
      self.collective_movement()
      self.sniffing()
      self.update_hierarchy() # remake True is good only to Multimodal
      if self.analytic_in:
        self.get_analytic_in(i, euclidean=False, volitive=True)
      self.optimum_score_tracking_iter.append(self.best.score)
      self.optimum_posit_tracking_iter.append(self.best.pos.tolist())

/content/drive/My Drive/POLI/decimo/tcc/Algoritms


### Register scores in each step volitive

In [ ]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for v in [0.0001, 0.001, 0.01, 0.1, 0.2, 0.4]:
    print (f"starting VGWO ({v} step volutive)")
    search_space_initializer = UniformSSInitializer()
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + "VGWO-vol" + os.sep
    num_exec = 1
    pack_size = 30
    num_iterations = 1000
    d = 15
    min_ai = 0.1
    step_volitive_init = v*10
    step_volitive_final = v

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction]
    regular_functions = unimodal_funcs + multimodal_funcs

    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + f"/{v}-vol.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['step_vol', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_iterations)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      start = time.time()
      bests_iter, bests_eval = run_experiments(num_iterations, pack_size, func,
                                              search_space_initializer,
                                              step_volitive_init,
                                              step_volitive_final, min_ai)
      end = time.time()
      row_csv = [v, func.function_name, (end - start)] + [b for b in bests_iter[:num_iterations]]
      writer_csv.writerow(row_csv)
      print(func.function_name, end - start, min(bests_iter))
      
    f_handle_csv.close()

def run_experiments(n_iter, pack_size, objective_function,
                    search_space_initializer, step_volitive_init,
                    step_individual_final, min_ai):
  opt1 = VolitivePack(objective_function=objective_function,
                      space_initializer=search_space_initializer,
                      n_iter=n_iter, pack_size=pack_size,
                      vol_final=step_individual_final, vol_init=step_volitive_init,
                      min_ai=min_ai)
  opt1.optimize()
  return opt1.optimum_score_tracking_iter, opt1.optimum_score_tracking_eval

main()

starting VGWO (0.0001 step volutive)
Sphere 7.983411073684692 1.0036306676725677e-98
RotatedHyperEllipsoid 5.969454288482666 5.4047225809831194e-105
Rosenbrock 3.2129552364349365 10.229702331274732
Dixon-Price 3.2869997024536133 0.0016841002894346128
Quartic-Noise 3.038925886154175 0.0007017564874704096
Generalized-Shwefel 2.92057204246521 3798.5225964484034
Rastrigin 3.0494308471679688 2.842170943040401e-14
Ackley 3.396374464035034 7.549516567451064e-15
Griewank 3.7725794315338135 0.0
Levi 4.8132710456848145 2.1598449493429825
starting VGWO (0.001 step volutive)
Sphere 3.203632354736328 2.493546372321448e-97
RotatedHyperEllipsoid 6.0104734897613525 9.665975893446953e-102
Rosenbrock 3.165703773498535 11.078946088162867
Dixon-Price 3.286334276199341 0.0003937141889866205
Quartic-Noise 3.0491719245910645 0.0016962828277839138
Generalized-Shwefel 2.8981308937072754 3429.6810642815626
Rastrigin 3.085036039352417 19.76298034893935
Ackley 3.3553409576416016 7.549516567451064e-15
Griewank 3.7

In [ ]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for v in [0.0001, 0.001, 0.01, 0.1, 0.2, 0.4]:
    print (f"starting VGWO ({v} step volutive)")
    search_space_initializer = UniformSSInitializer()
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + "VGWO-vol" +  os.sep
    num_exec = 30
    pack_size = 30
    num_iterations = 1000
    d = 15
    min_ai = 0.1
    step_volitive_init = v*10
    step_volitive_final = v

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction]
    regular_functions = unimodal_funcs + multimodal_funcs

    # regular_functions = [RastriginFunction, RosenbrockFunction]

    # Notice that for CEC Functions only the following dimensions are available:
    # 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100
    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + f"/VGWO_exec_{v}.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['step_vol', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_exec)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      
      start = time.time()
      runs = run_experiments(num_iterations, pack_size, num_exec, func,
                            search_space_initializer,step_volitive_init, 
                            step_volitive_final, min_ai)
      end = time.time()
      row_csv = [v, func.function_name, (end - start)] + [r for r in runs]
      writer_csv.writerow(row_csv)
      
    f_handle_csv.close()


def run_experiments(n_iter, pack_size, num_runs, objective_function,
                    search_space_initializer, step_volitive_init,
                    step_individual_final, min_ai):
  alg_name = "VGWO"
  console_out = "Algorithm: {} Function: {} Execution: {} Best Cost: {}"
  runs = []

  for run in range(num_runs):
    opt1 = VolitivePack(objective_function=objective_function,
                        space_initializer=search_space_initializer,
                        n_iter=n_iter, pack_size=pack_size,
                        vol_final=step_individual_final, vol_init=step_volitive_init,
                        min_ai=min_ai)
    
    opt1.optimize()
    alpha = min(opt1.optimum_score_tracking_iter)
    runs.append(alpha)
    print (console_out.format(alg_name, objective_function.function_name, run+1, alpha))

    temp_optimum_cost_tracking_iter = np.asmatrix(opt1.optimum_score_tracking_iter)
    temp_optimum_cost_tracking_eval = np.asmatrix(opt1.optimum_score_tracking_eval)

  return runs

main()

starting VGWO (0.0001 step volutive)
Algorithm: VGWO Function: Sphere Execution: 1 Best Cost: 3.1397683267703384e-102
Algorithm: VGWO Function: Sphere Execution: 2 Best Cost: 4.2287279675473515e-102
Algorithm: VGWO Function: Sphere Execution: 3 Best Cost: 1.2550126514457034e-98
Algorithm: VGWO Function: Sphere Execution: 4 Best Cost: 2.042023058964178e-102
Algorithm: VGWO Function: Sphere Execution: 5 Best Cost: 3.995590023387382e-101
Algorithm: VGWO Function: Sphere Execution: 6 Best Cost: 1.3318158612901842e-100
Algorithm: VGWO Function: Sphere Execution: 7 Best Cost: 2.17023209257689e-103
Algorithm: VGWO Function: Sphere Execution: 8 Best Cost: 1.0885878489878006e-99
Algorithm: VGWO Function: Sphere Execution: 9 Best Cost: 5.901486944676733e-101
Algorithm: VGWO Function: Sphere Execution: 10 Best Cost: 3.1233148339190812e-99
Algorithm: VGWO Function: Sphere Execution: 11 Best Cost: 2.302902134460223e-103
Algorithm: VGWO Function: Sphere Execution: 12 Best Cost: 4.041647901026525e-10

### Register performance curve VGWO in each iteration

Validate learning in each iteration

In [7]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30, 50, 100]:
    print (f"starting VGWO ({d})")
    search_space_initializer = UniformSSInitializer()
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + f"{d}d" + os.sep
    num_exec = 1
    pack_size = 30
    num_iterations = 1000
    min_ai = 0.1
    step_volitive_init = 0.1
    step_volitive_final = 0.01

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction]
    regular_functions = unimodal_funcs + multimodal_funcs

    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + "/VGWO_iter.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['opt', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_iterations)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      start = time.time()
      bests_iter, bests_eval = run_experiments(num_iterations, pack_size, func,
                                              search_space_initializer,
                                              step_volitive_init,
                                              step_volitive_final, min_ai)
      end = time.time()
      row_csv = ['VGWO', func.function_name, (end - start)] + [b for b in bests_iter[:num_iterations]]
      writer_csv.writerow(row_csv)
      print(func.function_name, end - start, min(bests_iter))
      
    f_handle_csv.close()

def run_experiments(n_iter, pack_size, objective_function,
                    search_space_initializer, step_volitive_init,
                    step_individual_final, min_ai):
  opt1 = VolitivePack(objective_function=objective_function,
                      space_initializer=search_space_initializer,
                      n_iter=n_iter, pack_size=pack_size,
                      vol_final=step_individual_final, vol_init=step_volitive_init,
                      min_ai=min_ai)
  opt1.optimize()
  return opt1.optimum_score_tracking_iter, opt1.optimum_score_tracking_eval

main()

starting VGWO (15)
Sphere 2.5382471084594727 1.3674713796209632e-100
RotatedHyperEllipsoid 4.568625211715698 1.9974192278397924e-101
Rosenbrock 2.4640231132507324 12.163136802638142
Dixon-Price 2.5004055500030518 0.013061988921835018
Quartic-Noise 2.2778894901275635 0.0007761579526752348
Generalized-Shwefel 2.3104398250579834 1441.823659713963
Rastrigin 2.351485013961792 0.0
Ackley 2.6368024349212646 7.549516567451064e-15
Griewank 2.7736551761627197 0.008728159913702349
Levi 3.6543478965759277 2.1598449493429825
starting VGWO (30)
Sphere 2.8224031925201416 1.5717756796171025e-66
RotatedHyperEllipsoid 6.913408517837524 2.8412028942426003e-67
Rosenbrock 3.0330002307891846 27.01039899990767
Dixon-Price 3.297386646270752 0.002577101560636781
Quartic-Noise 2.856813669204712 0.0024095113702353476
Generalized-Shwefel 2.6078991889953613 9538.710055534315
Rastrigin 2.941288471221924 9.691685367662217
Ackley 2.97727632522583 2.1760371282653068e-14
Griewank 3.1657867431640625 0.0
Levi 4.036438941

### Register bests and registration in 30 executions

Validate bests results in multiple sessions of iterations

In [8]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30, 50, 100]:
    print (f"starting VGWO ({d})")
    search_space_initializer = UniformSSInitializer()
    file_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Executions" + os.sep + f"{d}d" + os.sep
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + f"{d}d" + os.sep
    num_exec = 30
    pack_size = 30
    num_iterations = 1000
    min_ai = 0.1
    step_volitive_init = 0.1
    step_volitive_final = 0.01

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction]
    regular_functions = unimodal_funcs + multimodal_funcs

    # regular_functions = [RastriginFunction, RosenbrockFunction, AckleyFunction]

    # Notice that for CEC Functions only the following dimensions are available:
    # 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100
    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + "/VGWO_exec.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['opt', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_exec)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      if func.function_name != 'Perm':
      
        start = time.time()
        runs = run_experiments(num_iterations, pack_size, num_exec, func,
                              search_space_initializer,step_volitive_init, 
                              step_volitive_final, min_ai, file_path)
        end = time.time()
      else:
        start = time.time()
        end = time.time()
        runs = [1.8956733182600927e+307, 1.2772059494392159e+306, 1.4182018194829845e+308, 1.3361609307243766e+307, 1.285834762159742e+307, 1.736565747266942e+300, 8.453597932831765e+307, 1.1728657310360194e+308, 2.571598034858426e+305, 1.5120841137500091e+308,
                2.1929339749442197e+304, 1.6002261214723387e+304, 4.003282209148503e+302, 2.3597690260655865e+306, 3.8856379996953566e+302, 8.922127714600043e+306, 8.766100214902645e+305, 3.486014993849642e+307, 6.526971729168271e+307, 7.335281993306942e+305,
                3.547905908733193e+305, 4.1388463098618e+307, 4.2398824620203983e+300, 1.127629623436446e+299, 2.3089069452438043e+299, 8.397907534765697e+305, 1.7592763661295446e+305, 2.6526202826347347e+307, 1.6760145930608865e+308, 8.57241323977012e+305]
      row_csv = ['VGWO', func.function_name, (end - start)] + [r for r in runs]
      writer_csv.writerow(row_csv)
      
    f_handle_csv.close()


def run_experiments(n_iter, pack_size, num_runs, objective_function,
                    search_space_initializer, step_volitive_init,
                    step_individual_final, min_ai, save_dir):
  alg_name = "VGWO"
  console_out = "Algorithm: {} Function: {} Execution: {} Best Cost: {}"
  if save_dir:
    create_dir(save_dir)
    f_handle_cost_iter = open(save_dir + "/VGWO_" + objective_function.function_name + "_cost_iter.txt", 'w+')
    f_handle_cost_eval = open(save_dir + "/VGWO_" + objective_function.function_name + "_cost_eval.txt", 'w+')
    runs = []

  for run in range(num_runs):
    opt1 = VolitivePack(objective_function=objective_function,
                        space_initializer=search_space_initializer,
                        n_iter=n_iter, pack_size=pack_size,
                        vol_final=step_individual_final, vol_init=step_volitive_init,
                        min_ai=min_ai)
    
    opt1.optimize()
    alpha = min(opt1.optimum_score_tracking_iter)
    runs.append(alpha)
    print (console_out.format(alg_name, objective_function.function_name, run+1, alpha))

    temp_optimum_cost_tracking_iter = np.asmatrix(opt1.optimum_score_tracking_iter)
    temp_optimum_cost_tracking_eval = np.asmatrix(opt1.optimum_score_tracking_eval)

    if save_dir:
      np.savetxt(f_handle_cost_iter, temp_optimum_cost_tracking_iter, fmt='%.4e')
      np.savetxt(f_handle_cost_eval, temp_optimum_cost_tracking_eval, fmt='%.4e')

  if save_dir:
    f_handle_cost_iter.close()
    f_handle_cost_eval.close()

  return runs

main()

starting VGWO (15)
Algorithm: VGWO Function: Sphere Execution: 1 Best Cost: 6.955222356053754e-100
Algorithm: VGWO Function: Sphere Execution: 2 Best Cost: 5.2557177201377234e-102
Algorithm: VGWO Function: Sphere Execution: 3 Best Cost: 1.223755732858708e-100
Algorithm: VGWO Function: Sphere Execution: 4 Best Cost: 2.509956317665146e-99
Algorithm: VGWO Function: Sphere Execution: 5 Best Cost: 3.123791798995624e-101
Algorithm: VGWO Function: Sphere Execution: 6 Best Cost: 1.5965213215115691e-102
Algorithm: VGWO Function: Sphere Execution: 7 Best Cost: 3.8732488226561255e-100
Algorithm: VGWO Function: Sphere Execution: 8 Best Cost: 1.2818456222551222e-100
Algorithm: VGWO Function: Sphere Execution: 9 Best Cost: 3.2582847641599877e-100
Algorithm: VGWO Function: Sphere Execution: 10 Best Cost: 1.6432285744567973e-98
Algorithm: VGWO Function: Sphere Execution: 11 Best Cost: 3.140511600062114e-101
Algorithm: VGWO Function: Sphere Execution: 12 Best Cost: 5.09981541061467e-100
Algorithm: VGWO

### Creating data to simulate IN in each execution/simulation by each iterations

Here we go develop many executation with many iterations that save each information about alpha (scores in each iteration and last position), evolution score and iteration between each wolf.

Estudo do IN comparando o FSS com GWO e o que pode ser mesclado disso para o estudo de IN do VGWO:
 - No GWO só o trio lider influencia na matilha já que só existe o movimento coletivo e é graças a eles;
 - No FSS existe um cálculo para o a IN do movimento voltível separado o cálculo de IN para movimento coletivo;
 - O VGWO apesar de ter mais de um movimento (col e vol) como no FSS, o estudo da IN vai ser feito apenas a partir do resultado final da iteração dos lobos;
    - Vai ser com o resultado final para facilitar o estudo e se assemelhar ao GWO;
    - Vai utilizar o col e vol somados pela regra do GWO e FSS para determinar a influencia da rede;
    - O valor final da influencia dos dois movimentos equivale a influencia da rede naquela iteração.

In [ ]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30, 50, 100]:
    print (f"starting VGWO ({d})")
    search_space_initializer = UniformSSInitializer()
    file_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "IN" + os.sep + f"{d}d" + os.sep
    num_exec = 30
    pack_size = 30
    num_iterations = 1000
    min_ai = 0.1
    step_volitive_init = 0.001
    step_volitive_final = 0.000

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, PermFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction, Levi13Function]
    regular_functions = unimodal_funcs + multimodal_funcs

    # regular_functions = [AckleyFunction]

    # Notice that for CEC Functions only the following dimensions are available:
    # 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100
    cec_functions = []

    create_dir(file_path)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      f_handle_json = open(file_path + f"/VGWO_{func.function_name}_exec.json", 'w+', encoding='utf-8')
      
      run_json = run_experiments(num_iterations, pack_size, num_exec, func,
                            search_space_initializer,step_volitive_init, 
                            step_volitive_final, min_ai)

      json.dump(run_json, f_handle_json, ensure_ascii=False)
      f_handle_json.close()


def run_experiments(n_iter, pack_size, num_runs, objective_function,
                    search_space_initializer, step_volitive_init,
                    step_individual_final, min_ai):
  alg_name = "VGWO"
  console_out = "Algorithm: {} Function: {} Execution: {} Best Cost: {} IN: {}"
  runs = {"array": []}

  for run in range(num_runs):
    opt = VolitivePack(objective_function=objective_function,
                        space_initializer=search_space_initializer,
                        n_iter=n_iter, pack_size=pack_size,
                        vol_final=step_individual_final, vol_init=step_volitive_init,
                        min_ai=min_ai, analytic_in=True)
    opt.optimize()
    alpha = min(opt.optimum_score_tracking_iter)
    json_run = {
        "alpha_scores": opt.optimum_score_tracking_iter,
        "alpha_positions": opt.optimum_posit_tracking_iter,
        "in": opt.i_net.tolist(),
    }
    runs["array"].append(json_run)
    print(console_out.format(alg_name, objective_function.function_name, run+1, alpha, opt.i_net.shape))

    # temp_optimum_cost_tracking_iter = np.asmatrix(opt.optimum_score_tracking_iter)
    # temp_optimum_cost_tracking_eval = np.asmatrix(opt.optimum_score_tracking_eval)

  return runs

main()